In [4]:
class Square(object):
    def __init__(self, side):
        self.side = side
    def aget(self):
        return self.side ** 2
    def aset(self, value):
        print('Cannot set value.')
    def adel(self):
        print('Cannot del value.')
    area = property(aget, aset, adel, doc='Area of square')

In [5]:
square = Square(4)

In [6]:
square.area

16

In [26]:
# Inmmutable value
class Square(object):
    def __init__(self, side):
        self.side = side
    @property
    def area(self):
        ''' Calculate area'''
        return self.side * self.side
    @area.setter
    def area(self, value):
        print('cannot set value')
    @area.deleter
    def area(self):
        print('cannot delete')

In [21]:
def wrapper(func):
    print(locals())
    def inner(*args, **kwargs):
        result = func(*args, **kwargs)
        return result
    return inner

@wrapper
def hello(*args, **kwargs):
    return args, kwargs

hello('hey', 8)

{'func': <function hello at 0x7f462461bae8>}


(('hey', 8), {})

In [27]:
def nested_property(func):
    """ Simpler function definition """
    names = func()
    names['doc'] = func.__doc__
    return property(**names)
    
class Square(object):
    def __init__(self, side):
        self.side = side
        
    @nested_property
    def area():
        ''' property for area '''
        def fget(self):
            return self.side * self.side
        def fset(self, value):
            print('cannot set value.')
        def fdel(self):
            print('cannot del value')
            
        return locals()

In [32]:
s = Square(4)
s.area

16

In [33]:
class DataDescriptor(object):
    def __init__(self):
        self.value = 0
    def __get__(self, instance, cls):
        print('__get__')
        return self.value
    def __set__(self, instace, value):
        print('__set__')
        try:
            self.value = value.upper()
        except AttributeError:
            self.value = value
    def __delete__(self, instance):
        print('__delete__')

In [45]:
class A:
    attr = DataDescriptor()

In [55]:
a = A()
a.attr
a.attr = 'hello'
a.attr == type(a).__dict__['attr'].__get__(a, type(a))

__get__
__set__
__get__
__get__


True

In [48]:
A.attr
A.__dict__['attr'].__get__(None, A)

__get__
__get__


0

In [50]:
A.__dict__

mappingproxy({'__module__': '__main__',
              'attr': <__main__.DataDescriptor at 0x7f74a9975080>,
              '__dict__': <attribute '__dict__' of 'A' objects>,
              '__weakref__': <attribute '__weakref__' of 'A' objects>,
              '__doc__': None})

In [56]:
try:
    A.__dict__['a'] = 5
except TypeError:
    pass

In [58]:
a.x = 100
a.__dict__

{'x': 100}

In [61]:
class Overridden(object):
    attr = DataDescriptor()
    def __getattribute__(self, name):
        ''' Always called when accessing an attribute. '''
        print('no')

In [68]:
o = Overridden()
o.x = 10
o.attr

no


In [85]:
def func():
    pass
func.__get__
func.x = 10
func.__gt__?

In [72]:
class C(object):
    def method():
        pass
c = C()
c.method

<bound method C.method of <__main__.C object at 0x7f74a99024a8>>

In [74]:
C.method

<function __main__.C.method()>

In [86]:
type(c).__dict__['method'].__get__(c, type(c))

<bound method C.method of <__main__.C object at 0x7f74a99024a8>>

In [93]:
from weakref import WeakKeyDictionary
class DescriptorWeakKeyDictStorage(object):
    _hidden = WeakKeyDictionary()
    def __init__(self, default=None):
        self.default = default
        
    def __get__(self, instance, owner):
        return DescriptorWeakKeyDictStorage._hidden.get(
            instance, self.default)
    def __set__(self, instance, value):
        DescriptorWeakKeyDictStorage._hidden[instance] = value
class StoreInstance(object):
    attr = DescriptorWeakKeyDictStorage(10)
store1 = StoreInstance()
store2 = StoreInstance()
print(store1.attr)
print(store2.attr)
store1.attr = 100
print(store1.attr)
print(store2.attr)

print(dict(DescriptorWeakKeyDictStorage.__dict__['_hidden']))
del store1
print(dict(DescriptorWeakKeyDictStorage.__dict__['_hidden']))

10
10
100
10
{<__main__.StoreInstance object at 0x7f74a94ea9e8>: 100}
{}


In [95]:
class DescriptorDictStorage(object):
    _hidden = {}
    def __init__(self, default=None):
        self.default = default
        
    def __get__(self, instance, owner):
        return DescriptorDictStorage._hidden.get(
            instance, self.default)
    def __set__(self, instance, value):
        DescriptorDictStorage._hidden[instance] = value
class StoreInstance(object):
    attr = DescriptorDictStorage(10)
store1 = StoreInstance()
store2 = StoreInstance()
print(store1.attr)
print(store2.attr)
store1.attr = 100
print(store1.attr)
print(store2.attr)

print(dict(DescriptorDictStorage.__dict__['_hidden']))
del store1
print(dict(DescriptorDictStorage.__dict__['_hidden']))

10
10
100
10
{<__main__.StoreInstance object at 0x7f74a94eabe0>: 100}
{<__main__.StoreInstance object at 0x7f74a94eabe0>: 100}


In [96]:
from weakref import WeakKeyDictionary
class Checked(object):
    _hidden = WeakKeyDictionary()
    def __init__(self, checker=None, default=None):
        if checker:
            checker(default)
        self.checker = checker
        self.default = default
    def __get__(self, instance, owner):
        return Checked._hidden.get(instance, self.default)
    def __set__(self, instace, value):
        if self.checker:
            self.checker(value)
        Checked._hidden[instance] = value

def is_int(value):
    if not isinstance(value, int):
        raise ValueError('Must be int')

class Restricted(object):
    attr1 = Checked(checker=is_int, default=10)
    attr2 = Checked(default=2.5)
    try:
        attr3 = Checked(checker=is_int, default=12.5)
    except ValueError:
        attr3 = Checked(checker=is_int, default=12)

In [ ]:
class Immutable:
    def __init__(self, default):
        self.default = default
    
    def __get__(self, instance, owner):
        return self.default
    
    def __set__(self, instance, value):
        self.value = value

In [1]:
dict()

SyntaxError: invalid syntax (<ipython-input-1-3e803a8c91ec>, line 1)